# Plotly Viewer Examples
This notebook contains examples of how to use the viewers provided by the `glue-plotly` package

### Imports

In [1]:
from glue.core import Data
from glue_jupyter import jglue
from glue_plotly.viewers.histogram.viewer import PlotlyHistogramView
from glue_plotly.viewers.scatter.viewer import PlotlyScatterView

Create a glue-jupyter application and session

In [2]:
app = jglue()
session = app.session
dc = app.session.data_collection

Create some random data to use, and do a bit of styling

In [3]:
from random import randint
x = [randint(0, 100) for _ in range(100)]
y = [randint(300, 400) for _ in range(100)]
data = Data(label='data', x=x, y=y)
dc.append(data)
data.style.color = "#0000ff"
data.style.markersize = 6

In [4]:
x2 = [randint(0, 100) for _ in range(100)]
y2 = [randint(300, 400) for _ in range(100)]
data2 = Data(label='data2', x=x2, y=y2)
dc.append(data2)
app.add_link(data, 'x', data2, 'x')
app.add_link(data, 'y', data2, 'y')
data.style.color = "#0000ff"
data.style.markersize = 6

### Create a Plotly Scatter Viewer

In [5]:
scatter_viewer = app.new_data_viewer("plotly_scatter", data=data)
# Alternatively, you can use the viewer class:
# viewer = app.new_data_viewer(PlotlyScatterView, data=data)

LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(template=Template(template='<template>\n…

In [6]:
scatter_viewer.add_data(data2)

True

In [7]:
scatter_viewer.layers

(PlotlyScatterLayerArtist for data, PlotlyScatterLayerArtist for data2)

### Create a Plotly Histogram Viewer

In [8]:
scatter_viewer.layers[1].state.zorder

2

In [9]:
histogram_viewer = app.new_data_viewer("plotly_histogram", data=data)
# Alternatively, you can use the viewer class:
# viewer = app.new_data_viewer(PlotlyHistogramView, data=data)

LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(template=Template(template='<template>\n…

You can adjust whether there are gaps between the bars and, if so, what fraction of the plot they occupy

In [10]:
histogram_viewer.state.gaps = True
histogram_viewer.state.gap_fraction = 0.15

### Dotplot Viewer
`glue-plotly` provides an alternate layer artist for the histogram viewer that displays data as a dotplot. While we don't offer a built-in viewer that uses this functionality, you can use the simple implementation below as a starting point if you're interested in using dotplots.

In [103]:
from echo import delay_callback
from glue.config import viewer_tool
from glue_plotly.viewers.common.tools import PlotlyHZoomMode

@viewer_tool
class BinningHZoom(PlotlyHZoomMode):
    tool_id = "plotly:binhzoom"

    def _on_selection(self, trace, points, selector):
        xmin, xmax = selector.xrange
        viewer_state = self.viewer.state
        with self.viewer.figure.batch_update(), delay_callback(viewer_state, 'x_min', 'x_max'):
            viewer_state.x_min = xmin
            viewer_state.x_max = xmax
        self.viewer.state.update_bins_to_view()

In [104]:
from glue_plotly.viewers.histogram.dotplot_layer_artist import PlotlyDotplotLayerArtist

class DotplotViewer(PlotlyHistogramView):

    tools = ["plotly:binhzoom"]
    _data_artist_cls = PlotlyDotplotLayerArtist
    _subset_artist_cls = PlotlyDotplotLayerArtist

In [105]:
dotplot_viewer = app.new_data_viewer(DotplotViewer, data=data)

LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(template=Template(template='<template>\n…

In [108]:
# Check that the dotplot is displaying the correct values
values = dotplot_viewer.layers[0].layer[dotplot_viewer.state.x_att]
mask = (values >= dotplot_viewer.state.hist_x_min) & (values <= dotplot_viewer.state.hist_x_max)
print(dotplot_viewer.state.hist_x_min, dotplot_viewer.state.hist_x_max)
print(values[mask])
print(len(values[mask]))

27.638810597519733 33.05031003382187
[30 31 29 31 30 33]
6
